# Read PMaps


In [1]:
import datetime

In [2]:
from __future__ import print_function
import sys
import os
from glob import glob
from time import time

In [3]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import pandas as pd
import tables as tb
import numpy as np
import math

In [4]:
from invisible_cities.database import load_db
from invisible_cities.core.system_of_units_c import SystemOfUnits
import invisible_cities.sierpe.blr as blr
import invisible_cities.core.mpl_functions as mpl
import invisible_cities.core.wfm_functions as wfm
import invisible_cities.core.tbl_functions as tbl
import invisible_cities.core.peak_functions_c as cpf
import invisible_cities.core.pmaps_functions as pf
import invisible_cities.core.sensor_functions as sf
from invisible_cities.core.core_functions import define_window

In [5]:
import pylab as pyl

### Access to database and define units

In [6]:
DataPMT = load_db.DataPMT()
units = SystemOfUnits()
adc_to_pes = abs(DataPMT.adc_to_pes.values)
DataSiPM = load_db.DataSiPM()
adc_to_pes_sipm = DataSiPM.adc_to_pes.values
xs = DataSiPM.X.values
ys = DataSiPM.Y.values

In [7]:
#PMAP_file = os.environ['IC_DATA']  + '/pmaps_dst_NEXT_v0_08_06_Kr_ACTIVE_86_0_7bar_MCRD_10000.root.h5'
PMAP_file = os.environ['IC_DATA']+'/pmaps_JM/pmaps_NEXT_v0_08_09_Kr_ACTIVE_0_0_7bar__10000.root.h5'
#PMAP_file = os.environ['IC_DATA']+ '/pmaps_NEXT_v0_08_09_Kr_ACTIVE_0_0_7bar__10000.root.h5'

In [8]:
print(PMAP_file)

/Users/neus/InvCities/data/pmaps_JM/pmaps_NEXT_v0_08_09_Kr_ACTIVE_0_0_7bar__10000.root.h5


In [9]:
t = tb.open_file(PMAP_file,mode='r')

In [10]:
print(t)

/Users/neus/InvCities/data/pmaps_JM/pmaps_NEXT_v0_08_09_Kr_ACTIVE_0_0_7bar__10000.root.h5 (File) ''
Last modif.: 'Mon Feb 20 14:52:00 2017'
Object Tree: 
/ (RootGroup) ''
/PMAPS (Group) ''
/PMAPS/S1 (Table(51537,), shuffle, zlib(4)) 'S1 Table'
/PMAPS/S2 (Table(90642,), shuffle, zlib(4)) 'S2 Table'
/PMAPS/S2Si (Table(45075,), shuffle, zlib(4)) 'S2Si Table'



In [11]:
S2Si_ = t.root.PMAPS.S2Si

## pyTable

In [12]:
S2Si_

/PMAPS/S2Si (Table(45075,), shuffle, zlib(4)) 'S2Si Table'
  description := {
  "event": Int32Col(shape=(), dflt=0, pos=0),
  "evtDaq": Int32Col(shape=(), dflt=0, pos=1),
  "peak": UInt8Col(shape=(), dflt=0, pos=2),
  "nsipm": Int16Col(shape=(), dflt=0, pos=3),
  "nsample": Int16Col(shape=(), dflt=0, pos=4),
  "ene": Float32Col(shape=(), dflt=0.0, pos=5)}
  byteorder := 'little'
  chunkshape := (3855,)
  autoindex := True
  colindexes := {
    "event": Index(6, medium, shuffle, zlib(1)).is_csi=False}

In [13]:
S2Si_[0:10]

array([(2, 2, 0, 784, 1, 19.937114715576172),
       (2, 2, 0, 784, 2, 36.92438507080078),
       (2, 2, 0, 784, 3, 41.82916259765625),
       (2, 2, 0, 784, 4, 21.731544494628906),
       (3, 3, 0, 274, 2, 25.915727615356445),
       (3, 3, 0, 274, 3, 21.248924255371094),
       (3, 3, 0, 281, 1, 28.683107376098633),
       (3, 3, 0, 281, 2, 40.70846939086914),
       (3, 3, 0, 281, 3, 37.875267028808594),
       (3, 3, 0, 282, 1, 35.865325927734375)], 
      dtype=[('event', '<i4'), ('evtDaq', '<i4'), ('peak', 'u1'), ('nsipm', '<i2'), ('nsample', '<i2'), ('ene', '<f4')])

In [14]:
len(S2Si_.col('event'))

45075

### Select ev info and create a list of SiPMs with signal

In [15]:
sipm_l = []
evSel = 2
for i in range(len(S2Si_.col('event'))):
    if(S2Si_.col('event')[i] == evSel+1):
        break
    print(i)
    print('event: {0}, sipm = {1}, nsample = {2}, ene = {3}'.format(S2Si_.col('event')[i], S2Si_.col('nsipm')[i],S2Si_.col('nsample')[i] ,S2Si_.col('ene')[i]))
    sipm_l.append(S2Si_.col('nsipm')[i])

0
event: 2, sipm = 784, nsample = 1, ene = 19.937114715576172
1
event: 2, sipm = 784, nsample = 2, ene = 36.92438507080078
2
event: 2, sipm = 784, nsample = 3, ene = 41.82916259765625
3
event: 2, sipm = 784, nsample = 4, ene = 21.731544494628906


In [16]:
print(sipm_l)

[784, 784, 784, 784]


In [17]:
mydf = pf.read_pmaps(PMAP_file)
mydf[1]

,event,evtDaq,peak,time,ene
0,0,0,0,619586.250000,73.247719
1,0,0,0,620637.500000,146.347244
2,0,0,0,621637.500000,323.282196
3,0,0,0,622637.500000,625.912231
4,0,0,0,623637.500000,986.156494
5,0,0,0,624637.500000,1201.956421
6,0,0,0,625637.500000,1024.452881
7,0,0,0,626637.500000,820.801147
8,0,0,0,627637.500000,534.381592
9,0,0,0,628637.500000,319.071411


In [18]:
S2 = pf.s12df_to_s12l(mydf[1])

In [19]:
#S2

In [24]:
def my_sipm_s2_dict(SIPM, S2d, thr=5 * units.pes):
    """Given a vector with SIPMs (energies above threshold), and a
    dictionary of S2s, S2d, returns a dictionary of SiPMs-S2.  Each
    index of the dictionary correspond to one S2 and is a list of np
    arrays. Each element of the list is the S2 window in the SiPM (if
    not zero)
    """
    return {i: sipm_s2(SIPM, S2, thr=thr) for i, S2 in S2d.items()}

In [25]:
def sipm_s2(dSIPM, S2, thr=5*units.pes):
    """Given a vector with SIPMs (energies above threshold), return a list
    of np arrays. Each element of the list is the S2 window in the
    SiPM (if not zero).
    """
    #import pdb; pdb.set_trace()

    i0, i1 = index_from_s2(S2)
    dim = (i1 - i0)
    SIPML = []
    for i in dSIPM.keys():
        sipm = dSIPM[i][1]
        psum = np.sum(sipm[i0:i1])
        if psum > thr:
            e = np.zeros(dim, dtype=np.double)
            e[:] = sipm[i0:i1]
            SIPML.append([dSIPM[i][0], e])
    return SIPML

In [26]:
def index_from_s2(S2):
    """Return the indexes defining the vector."""
    T = S2[0] // units.mus
    return int(T[0]), int(T[-1]) + 1


In [23]:
my_sipm_s2_dict(sipm_l,S2)

TypeError: unsupported operand type(s) for //: 'list' and 'float'

## Panda table

#### Convert structured or record ndarray to DataFrame:

In [ ]:
S2Sidf = pd.DataFrame.from_records(S2Si_.read())

In [ ]:
#S2Sidf

In [ ]:
S2Sidf[:]

In [ ]:
t.root.PMAPS


In [ ]:
t.root.PMAPS.S2Si

In [ ]:
#t.root.PMAPS.S2Si[0:40]

In [ ]:
t.root.PMAPS.S1[:]

In [ ]:
t.root.PMAPS.S2[:]

In [ ]:
#pf.S12df_to_dict_select_peak()---------> on going
#sipm_s2_dict

In [ ]:
#pf.plot_s2si_map(S2Si_)

In [ ]:
t.root.PMAPS.S2Si[:]

In [ ]:
t.root.PMAPS.S2[0:1]

In [ ]:
t.root.PMAPS.S1[0:1]

# SiPMs inspection

In [ ]:
t.root.PMAPS.S2Si[0]

In [ ]:
datasetS2Si=f.root.PMAPS.S2Si
datasetS2=f.root.PMAPS.S2

In [ ]:
arr1ev=datasetS2Si[:]

In [ ]:
print(arr1ev)

In [ ]:
print(arr1ev[6])

In [ ]:
y=datasetS2Si.col('ene')
print(y)
len(y)

### Rango del narray

In [ ]:
range(len(y))

In [ ]:
x=datasetS2Si.col('event') 
print(x)

In [ ]:
x1=arr1ev[6]
print(x1)

In [ ]:
plt.hist(y, 10, histtype="bar", alpha=0.75)

In [ ]:
mpl.histo(y,10,"","","")

In [ ]:
#f.close()
#f

In [ ]:
#f=pf.read_pmaps(os.environ['IC_DATA'],'/pmaps_dst_NEXT_v0_08_06_Kr_ACTIVE_86_0_7bar_MCRD_10000.root.h5')

In [ ]:
datasetS2Si[0:20]

In [ ]:
pf.sipm_s2(datasetS2Si[0],datasetS2[0])

In [ ]:
datasetS2Si[0]

# PMTs inspection
## proves

In [ ]:
datasetS2=f.root.PMAPS.S2

In [ ]:
datasetS2

In [ ]:
x=datasetS2.col('event')

In [ ]:
len(x)

In [ ]:
x=datasetS2[0:20]
x